In [1]:
from collections import OrderedDict
from bokeh.models import ColumnDataSource, HoverTool
import bokeh.plotting as bk
import numpy as np
from math import pi
import pandas as pd

bk.output_notebook()

Loading BokehJS ...

Firstly, do some data processing on the Seattle file to prepare the content for the two charts.

In [7]:
data = pd.read_csv("data/seattle_incidents_summer_2014.csv")

jun_filter = data["Month"] == 6
jul_filter = data["Month"] == 7
aug_filter  = data["Month"] == 8

# prepare for the 1st plot: 
# a histogram of the monthly totals

total_jun = len(data[jun_filter])
total_jul = len(data[jul_filter])
total_aug = len(data[aug_filter])

In [8]:
# prepare for the 2nd plot

grouped = data.groupby(data["Offense Code"])

mean_counts = data["Offense Code"].value_counts() / 3
# at this point the codes (index) are ordered by average monthly incidents

# join further details to this column without changing the order
counts_jun = data["Offense Code"][jun_filter].value_counts()
counts_jul = data["Offense Code"][jul_filter].value_counts()
counts_aug = data["Offense Code"][aug_filter ].value_counts()

joint = pd.concat([mean_counts, counts_jun, counts_jul, counts_aug], 
                  axis=1, join_axes=[mean_counts.index])

joint.columns = ["c_mean", "c_jun", "c_jul", "c_aug"]

ranks = range(1, len(joint) + 1)

codes = joint.index

In [26]:
# construct the 1st plot

TOOLS = "hover,save"
source = ColumnDataSource(data=dict(month=["June", "July", "August"]))
                          
p1 = bk.figure(title = "Summer 2014 crime breakdown in Seattle by month", 
               background_fill_color="#DDDDDD", width=900, height=600, 
               tools=TOOLS)

p1.quad(top = [total_jun, total_july, total_aug], bottom=0, 
        left=[0, 1, 2], right=[1, 2, 3],
        fill_color=["#AA2222", "#22AA22", "#2222AA"],
        source=source
       )

hover = p1.select(dict(type=HoverTool))

hover.tooltips = OrderedDict([
        ("Month", "@month")
    ])

In [17]:
# construct the 2nd plot

TOOLS = "pan,box_zoom,resize,wheel_zoom,reset,hover,save"
p2 = bk.figure(title="Summer 2014 crime breakdown in Seattle by offense code", tools=TOOLS,
               plot_height=600, plot_width=900)

source_mean = ColumnDataSource(data=dict(rank=ranks,
                                         count=joint.c_mean,
                                         code=codes))
source_jun = ColumnDataSource(data=dict(rank=ranks,
                                        count=joint.c_jun,
                                        code=codes))
source_jul = ColumnDataSource(data=dict(rank=ranks,
                                        count=joint.c_jul,
                                        code=codes))
source_aug = ColumnDataSource(data=dict(rank=ranks,
                                        count=joint.c_aug,
                                        code=codes))

p2.line(ranks, joint.c_mean, color="#222222", line_width=2, name="foo", 
        source=source_mean, legend="Monthly mean")
p2.line(ranks, joint.c_jun, color="#AA2222", line_width=2, name="foo2", 
        source=source_jun, legend="June incidents")
p2.line(ranks, joint.c_jul, color="#22AA22", line_width=2, name="foo3", 
        source=source_jul, legend="August incidents")
p2.line(ranks, joint.c_aug, color="#2222AA", line_width=2, name="foo4", 
        source=source_aug, legend="July incidents")

hover = p2.select(dict(type=HoverTool))

hover.tooltips = OrderedDict([
        ("Rank", "@rank"),
        ("Incidents", "@count"),
        ("Offense Code", "@code")
    ])



In [27]:
bk.show(p1)
bk.show(p2)